## Capstone 1 - Part 2

In the first part of the capstone, we focused on Data Retrieval, Data Preprocessing, Feature Engineering and Exploratory Data Analysis using Python & its libraries. Now we are going to shift gears and gain insights into our HR Analytics dataset using SQL.

## TODO: Make use of SQL to do the following:

### Create a SQLITE3 DB using the CSV file (2 pts). Please refer this [link](https://www.linkedin.com/pulse/accessing-sqlite3-database-from-jupyter-notebook-using-varun-lobo/) and this [link](https://www.geeksforgeeks.org/how-to-import-csv-file-in-sqlite-database-using-python/) to know more.

### Calculate the Attrition Rate and summarize attrition (3 pts) by:
- Gender
- Department
- Age
- Average monthly income by job level
- Years at company

### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

### Effective Communication (2 pts)
- Please make use of markdown cells to communicate your thought process, why did you think of performing a step? what was the observation from the query? etc.
- The code should be commented so that it is readable for the reviewer.

### Grading and Important Instructions
- Each of the above steps are mandatory and should be completed in good faith
- Make sure before submitting that the code is in fully working condition
- It is fine to make use of ChatGPT, stackoverflow type resources, just provide the reference links from where you got it
- Debugging is an art, if you find yourself stuck with errors, take help of stackoverflow and ChatGPT to resolve the issue and if it's still unresolved, reach out to me for help.
- You need to score atleast 7/10 to pass the project, anything less than that will be marked required, needing resubmission.
- Feedback will be provided on 3 levels (Awesome, Suggestion, & Required). Required changes are mandatory to be made.
- For submission, please upload the project on github and share the link to the file with us through LMS.

#### Creating SQLite3 DB using CSV file

In [137]:
import sqlite3
import pandas as pd

df=pd.read_csv('/Users/yuti/Documents/Elevate Me/Fundamental/SQL /Capstone Part 1/HR-Analytics Dataset.csv')

# Connecting to SQLite3 database 
conn = sqlite3.connect('HR_Analytics_Database.db')
cursor = conn.cursor()




In [22]:
# Inserting DataFrame into SQLite table, creating the table automatically
df.to_sql('HR_Analytics', conn, if_exists='replace', index=False)

# Commit and close the connection
conn.commit()

In [23]:
# Query the table to ensure data has been inserted
query = "SELECT * FROM HR_Analytics LIMIT 5;"
tables = pd.read_sql_query(query, conn)

# Display the list of tables in the database
tables

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


#### Calculate Attrition Rate


In [41]:
query_attrition_rate = """
SELECT 
    Attrition,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM HR_Analytics), 1) AS Attrition_rate
FROM HR_Analytics
GROUP BY Attrition;
"""

# Execute the query and fetch the result
attrition_rate = pd.read_sql_query(query_attrition_rate, conn)

# Display the result
attrition_rate

,Attrition,Attrition_rate
0,No,83.9
1,Yes,16.1


#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857886?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NiwiaG1hYyI6ImUyNGU0YWRhNWQwMDkxNWE2NWNiY2EwNGFlNDNiMTBlYmYyNWY0YzAiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Gender

In [52]:
query_attrition_by_gender = """SELECT 
    Attrition,
    Gender,
    COUNT(*) AS Count_gender,
     ROUND(COUNT(*) * 100.0 /SUM(COUNT(*)) OVER(PARTITION BY Gender),1) AS Attrition_by_gender
FROM HR_Analytics
GROUP BY Attrition,Gender;
"""
# Execute the query and fetch the result
attrition_by_gender = pd.read_sql_query(query_attrition_by_gender, conn)

# Display the result
attrition_by_gender


,Attrition,Gender,Count_gender,Attrition_by_gender
0,No,Female,501,85.2
1,Yes,Female,87,14.8
2,No,Male,732,83.0
3,Yes,Male,150,17.0


#### Sample output

![image](https://api-v4.skyprepapp.com/public_api/da/857882?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MiwiaG1hYyI6IjM1NDU5N2E4MjZmMTMxNWFhMTA4NzFiYTFkZjQ5YjBhNTYyOWYwMzgiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Dept

In [73]:
query_department_attrition = """SELECT 
    Attrition,
    Department,
    COUNT(*) AS Department_Attrition
FROM HR_Analytics
GROUP BY Attrition,Department;
"""
# Execute the query and fetch the result
department_attrition = pd.read_sql_query(query_department_attrition, conn)

# Display the result
department_attrition


,Attrition,Department,Department_Attrition
0,No,Human Resources,51
1,No,Research & Development,828
2,No,Sales,354
3,Yes,Human Resources,12
4,Yes,Research & Development,133
5,Yes,Sales,92


#### Sample Output

<img src="https://api-v4.skyprepapp.com/public_api/da/857884?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NCwiaG1hYyI6IjdmOTA4NzJjODZmYTgxOGUyMzkyYTBlZjhjYjljNjA4ZGM3NjkzMWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9" width="450" height="450">

#### Find Attrition by Age Groups

In [100]:
attrition_by_age = """
WITH age_group_cte AS(
SELECT Attrition,
CASE
    WHEN Age<30 THEN 'UNDER 30'
    WHEN Age>=30 AND Age<40 THEN '30-40'
    WHEN Age>=40 AND Age<50 THEN '40-50'
    WHEN Age>=50  THEN '50 & Over'
END
 AS Age_Group
FROM HR_Analytics
)

SELECT Attrition, Age_Group, Count(*) AS num,
ROUND(COUNT(*) * 100.0/ SUM(COUNT(*)) OVER(PARTITION BY Age_Group),2) AS percent_by_age
FROM age_Group_cte
GROUP BY Attrition, Age_Group;
""" 

# Execute the query and fetch the result
attrition = pd.read_sql_query(attrition_by_age, conn)

# Display the result
attrition


,Attrition,Age_Group,num,percent_by_age
0,No,30-40,533,85.69
1,Yes,30-40,89,14.31
2,No,40-50,315,90.26
3,Yes,40-50,34,9.74
4,No,50 & Over,150,86.71
5,Yes,50 & Over,23,13.29
6,No,UNDER 30,235,72.09
7,Yes,UNDER 30,91,27.91


#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857885?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NSwiaG1hYyI6IjcwNGVlOWIwYzg1MmMwNmNhODg0NmYyZDFlNWE3OTU1MTFhMGVmYWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Monthly Income

In [89]:
attrition = """
SELECT 
    Department,
    JobLevel,
    AVG(MonthlyIncome) AS avg_income,
    ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome END),1) AS attrition_avg_income,
    ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome END)- AVG(MonthlyIncome),1) AS difference
FROM HR_Analytics
GROUP BY Department, JobLevel;
"""
# Execute the query and fetch the result
attrition_avg_income = pd.read_sql_query(attrition, conn)

# Display the result
attrition_avg_income


,Department,JobLevel,avg_income,attrition_avg_income,difference
0,Human Resources,1,2733.212121,2415.7,-317.5
1,Human Resources,2,5563.461538,NaN,NaN
2,Human Resources,3,9623.000000,10216.0,593.0
3,Human Resources,4,16147.500000,NaN,NaN
4,Human Resources,5,19197.857143,NaN,NaN
5,Research & Development,1,2840.064516,2687.4,-152.7
6,Research & Development,2,5291.238434,5372.0,80.8
7,Research & Development,3,10170.488372,9503.8,-666.6
8,Research & Development,4,15634.676471,12169.0,-3465.7
9,Research & Development,5,19218.510204,19550.0,331.5


#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857883?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MywiaG1hYyI6Ijg3NTU4ZDU1ZjRjN2U1YWI3ODQzYjM1NzFkNjBjMjEwNGY5NWI5ODUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Years At Company

In [178]:
attrition_by_years = """
WITH years AS (
SELECT YearsAtCompany,Attrition,
        CASE
            WHEN YearsAtCompany < 2 THEN 'New Hires'
            WHEN YearsAtCompany >= 2 AND YearsAtCompany <= 5 THEN '2-5 years'
            WHEN YearsAtCompany >= 6 AND YearsAtCompany <= 10 THEN '6-10 years'
            WHEN YearsAtCompany >= 11 AND YearsAtCompany <= 20 THEN '11-20 years'
            WHEN YearsAtCompany > 20 THEN 'Over 20 years'
        END AS tenure_years
    FROM HR_Analytics
)

SELECT 
    tenure_years, 
    COUNT(CASE WHEN Attrition = 'Yes' THEN 1 END)  AS num,
    ROUND(COUNT(CASE WHEN Attrition = 'Yes' THEN 1 END)*100.0/SUM(COUNT(CASE WHEN Attrition = 'Yes' THEN 1 END)) OVER(),1) as Attrition_rate
FROM years
GROUP BY tenure_years

"""
# Execute the query and fetch the result
attrition= pd.read_sql_query(attrition_by_years, conn)

# Display the result
attrition




,tenure_years,num,Attrition_rate
0,11-20 years,12,5.1
1,2-5 years,87,36.7
2,6-10 years,55,23.2
3,New Hires,75,31.6
4,Over 20 years,8,3.4


#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857881?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MSwiaG1hYyI6IjFhYmY3NGI4MzQ1NzViMWZkNDJlMjcwYTUyOTQ0OWQwZjJjMjhmNWUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

# Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?


In [193]:
attrition_by_age = """
WITH age_group_cte AS(
SELECT Attrition, WorkLifeBalance,YearsSinceLastPromotion,PercentSalaryHike,
CASE
   
    WHEN Age>=40 AND Age<50 THEN '40-50'
    WHEN Age>=50  THEN '50 & Over'
    ELSE 'Other'
END
 AS Age_Group
FROM HR_Analytics
)

SELECT Attrition, Age_Group, Count(*) AS num,
ROUND(COUNT(*) * 100.0/ SUM(COUNT(*)) OVER(PARTITION BY Age_Group),2) AS percent_by_age,
AVG(WorkLifeBalance) as Avg_WorkLifeBalance,AVG(YearsSinceLastPromotion) as Avg_YearsSinceLastPromotion,AVG(PercentSalaryHike) as Avg_PercentSalaryHike
FROM age_Group_cte
GROUP BY Attrition, Age_Group;
""" 

# Execute the query and fetch the result
attrition = pd.read_sql_query(attrition_by_age, conn)

# Display the result
attrition


,Attrition,Age_Group,num,percent_by_age,Avg_WorkLifeBalance,Avg_YearsSinceLastPromotion,Avg_PercentSalaryHike
0,No,40-50,315,90.26,2.838095,2.761905,15.314286
1,Yes,40-50,34,9.74,2.558824,2.911765,14.500000
2,No,50 & Over,150,86.71,2.660000,3.240000,15.313333
3,Yes,50 & Over,23,13.29,2.608696,3.739130,14.739130
4,No,Other,768,81.01,2.781250,1.821615,15.180990
5,Yes,Other,180,18.99,2.683333,1.533333,15.255556


The analysis suggests that work-life balance, promotion opportunities, and career progression could be driving attrition among older employees (50 & Over). Employees who stayed generally report better work-life balance and fewer years since their last promotion, whereas employees who left had a lower work-life balance and longer times since promotions.

# - Why do people with higher pay still leave the company?


In [212]:
attrition_by_age = """

WITH salary_group_cte AS(
SELECT Attrition,MonthlyIncome,PercentSalaryHike,PerformanceRating,YearsSinceLastPromotion,
CASE
   
    WHEN MonthlyIncome>=1000 AND MonthlyIncome<5000 THEN 'Entry-Level Salary'
    WHEN MonthlyIncome>=5000 AND MonthlyIncome<10000  THEN 'Mid-Level Salary'
    WHEN MonthlyIncome>=10000 AND MonthlyIncome<15000  THEN 'Experienced Salary' 
    WHEN MonthlyIncome>=15000 AND MonthlyIncome<20000  THEN 'Senior-Level Salary'
END
 AS Salary_Group
FROM HR_Analytics
)

SELECT Attrition, Salary_Group, Count(*) AS num,
ROUND(COUNT(*) * 100.0/ SUM(COUNT(CASE WHEN Attrition = 'Yes' THEN 1 END)) OVER(PARTITION BY Salary_Group),1) AS percent_by_salary,
AVG(PercentSalaryHike) as Avg_PercentSalaryHike,AVG(YearsSinceLastPromotion) as Avg_YearsSinceLastPromotion,AVG(PerformanceRating) as Avg_PerformanceRating
FROM salary_Group_cte
GROUP BY Attrition, Salary_Group;
""" 

# Execute the query and fetch the result
attrition = pd.read_sql_query(attrition_by_age, conn)

# Display the result
attrition


,Attrition,Salary_Group,num,percent_by_salary,Avg_PercentSalaryHike,Avg_YearsSinceLastPromotion,Avg_PerformanceRating
0,No,Entry-Level Salary,586,359.5,15.267918,1.395904,3.155290
1,Yes,Entry-Level Salary,163,100.0,15.447853,1.257669,3.190184
2,No,Experienced Salary,128,640.0,15.546875,4.054688,3.179688
3,Yes,Experienced Salary,20,100.0,13.000000,5.900000,3.000000
4,No,Mid-Level Salary,391,798.0,15.125320,2.153453,3.138107
5,Yes,Mid-Level Salary,49,100.0,14.938776,2.081633,3.122449
6,No,Senior-Level Salary,128,2560.0,15.070312,4.500000,3.164062
7,Yes,Senior-Level Salary,5,100.0,13.600000,7.200000,3.000000
